In [1]:
# This command installs the necessary Python libraries.
# - ollama: The official client library to communicate with our local Ollama server.
# - pandas: A powerful library for loading and working with data from our CSV file.


%pip install -q ollama pandas

import ollama
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [2]:
def load_model(model_name):
    print(f"Sending a test message to the local model: '{model_name}'...")
    print("Please wait, the first response can take a moment...")
    try:
        # We send a simple chat message to the model.
        response = ollama.chat(
            model=model_name,
            messages=[
                {'role': 'user', 'content': 'In one short sentence, introduce yourself.'},
                ],
                think=False,
        )
        # If successful, we print the text content from the model's response.
        message_content = response['message']['content']
        print("\n Connection Successful! Model responded:")
        print(f' "{message_content}"')

    except Exception as e:
        print(f"\n An error occurred. Is Ollama running?")
        print(f" Please ensure the Ollama application is running on your computer.")
        print(f" Error details: {e}")



In [3]:
def load_filenet_errors(file_name):
    try:
        df = pd.read_csv(file_name)
        # --- Data Cleaning Best Practice ---
        # Remove any leading/trailing whitespace from column names.
        df.columns = df.columns.str.strip()
        # display() provides a nice, table-like output in Jupyter.
        display(df.head())

    except FileNotFoundError:
        print(f"Error: The file '{file_name}' was not found.")
        print(" Please make sure the CSV file is in the same folder as this Jupyter Notebook.")

    except Exception as e:
        print(f"An error occurred: {e}")
    return df

df=load_filenet_errors("filenet-errors.csv")


,ErrorCode,ErrorName,ErrorMessage
0,FNRAC1001W,WARN_TRACE_LOG_ENABLED,Server trace logging is enabled.
1,FNRAC1002W,WARN_ADDON_CANNOT_BE_ UNSELECTED,The selection of this add-on feature cannot be...
2,FNRAC1001E,ERR_UNEXPECTED,An unexpected error occurred.
3,FNRAC1002E,ERR_LOGIN_AUTHENTICATION,The system cannot log you on right now.
4,FNRAC1003E,ERR_LOGIN_ACTIVITY,You are no longer logged on.


In [4]:
# This template is our master instruction set for the Granite model.

prompt_template = """
**Instruction:**
You are an expert IBM FileNet P8 senior system administrator. Your task is to write a detailed, step-by-step troubleshooting tutorial for a junior administrator. Use the provided error details to create a realistic use-case scenario and a clear guide.

**Input Data:**
- Error Code: {error_code}
- Error Name: {error_name}
- Error Message: {error_message}

**Required Output Format (Strictly use Markdown):**

## Troubleshooting Guide: {error_code}

### 1. Understanding the Error

Provide a brief, scenario-based explanation of what this error means for a junior administrator.

### 2. Common Scenario

Describe a realistic, day-to-day situation where this error might occur. For example: "A user from the finance team is trying to add a new invoice to the system but receives this error..."

### 3. Step-by-Step Troubleshooting

1. **First Actionable Step:** (e.g., "Verify User Permissions in ACCE"). Explain how to do this and why it's important for this specific error.
2. **Second Actionable Step:** (e.g., "Check Server Logs"). Specify which logs to check (e.g., p8_server_error.log) and what keywords to look for.
3. **Third Actionable Step:** (e.g., "Review Class Definitions or Folder Security"). Explain the process clearly.


### 4. Summary of Solutions

Briefly summarize the most likely solutions to resolve the error.

"""



In [5]:
def generate_guide(model_name, error_code, error_message, error_name):
    """
    Generates a single troubleshooting guide by calling the local Ollama model.
    Args:
            error_code (str): The error code from our DataFrame.
            error_message (str): The short error message.
            explanation (str): The detailed explanation of the error.
    Returns:
            str: The AI-generated troubleshooting guide in Markdown format.
    """

    # Step 1: Fill in the placeholders in our template with the specific error data.
    full_prompt = prompt_template.format(
            error_code=error_code,
            error_name=error_name,
            error_message=error_message
    )

    # Step 2: Send the complete prompt to our custom-built local model.
    try:
        response = ollama.chat(
                model=model_name, # Our custom model name
                messages=[
                        {'role': 'user', 'content': full_prompt},
                ],
            think=False,
        )

       # Step 3: Extract and return just the text content from the model's response.
        return response['message']['content']

    except Exception as e:
        error_text = f" An error occurred while communicating with the Ollama model: {e}"
        print(error_text)
        return error_text


In [6]:
def generate_response(model_name, df, user_input):
    # 3. Search for the exact error code in the DataFrame (case-insensitive).
    result_df = df[df['ErrorCode'].str.lower() == user_input.lower()]

    # 4. --- SUCCESS PATH --- If a match is found:
    if not result_df.empty:
        # Get the first matching row from our data.
        error_details = result_df.iloc[0]
        print(f"\n Found '{error_details['ErrorCode']}'. Generating guide with AI, please wait...")

        print("-" * 35)
        # Call our AI function to generate the guide.
        guide_markdown = generate_guide(
            model_name=model_name,
            error_code=error_details['ErrorCode'],
            error_name=error_details['ErrorName'],
            error_message=error_details['ErrorMessage']
        )  

        # Display the formatted result in the notebook.
        display(Markdown(guide_markdown))
        print("-" * 35)
        return True

    # 5. --- FAILURE PATH --- If no exact match is found:
    else:
        print(f"\n Error: The code '{user_input}' was not found.")

        # BONUS: Suggest similar codes based on the user's input.
        suggestions = df[df['ErrorCode'].str.contains(user_input.upper())]
        if not suggestions.empty:
            print("\nDid you mean one of these?")
            for code in suggestions['ErrorCode'].tolist():
                print(f"- {code}")
        print("-" * 35)
        return False


In [7]:
df

,ErrorCode,ErrorName,ErrorMessage
0,FNRAC1001W,WARN_TRACE_LOG_ENABLED,Server trace logging is enabled.
1,FNRAC1002W,WARN_ADDON_CANNOT_BE_ UNSELECTED,The selection of this add-on feature cannot be...
2,FNRAC1001E,ERR_UNEXPECTED,An unexpected error occurred.
3,FNRAC1002E,ERR_LOGIN_AUTHENTICATION,The system cannot log you on right now.
4,FNRAC1003E,ERR_LOGIN_ACTIVITY,You are no longer logged on.


In [8]:

from IPython.display import display, Markdown

def prompt_user( model_name="ibm/granite4:latest"):
    
    load_model(model_name)
    print("--- IBM FileNet AI Troubleshooter ---")
    print("Enter a FileNet Error Code to generate a guide.")
    print("Type 'exit' or 'quit' to stop the tool.")
    print("-" * 35)

    # This loop will run continuously until the user types 'exit' or 'quit'.
    while True:
        # 1. Get input from the user.
        user_input = input("\nEnter the FileNet Error Code: ")
        # 2. Check if the user wants to exit.
        if user_input.lower() in ['exit', 'quit', '']:
            print("\nExiting the troubleshooter. Goodbye!")
            break

        if generate_response(model_name, df, user_input):
            break


In [9]:
def load_and_generate(model_name="ibm/granite4:latest", error_code="FNRAC1001W"):
    load_model(model_name)
    generate_response(model_name, df, error_code)


In [16]:
%%time
model_name="ibm/granite4:latest"
load_model(model_name)

Sending a test message to the local model: 'ibm/granite4:latest'...
Please wait, the first response can take a moment...

 Connection Successful! Model responded:
 "I am an AI assistant designed to provide accurate and reliable information while ensuring the safety of all users."
CPU times: user 3.22 ms, sys: 2.49 ms, total: 5.72 ms
Wall time: 560 ms


In [17]:
%%time
generate_response(model_name, df, "FNRAC1001W")


 Found 'FNRAC1001W'. Generating guide with AI, please wait...
-----------------------------------


## Troubleshooting Guide: FNRAC1001W

### 1. Understanding the Error

The error code **FNRAC1001W** with the name **WARN_TRACE_LOG_ENABLED** and message "**Server trace logging is enabled**" indicates that the server-side tracing for IBM FileNet P8 has been activated. This can happen due to various reasons, such as a misconfiguration in the system settings or an application attempt to log detailed information that requires elevated permissions.

For a junior administrator, encountering this error typically suggests that there might be issues with logging configurations, which could affect debugging and troubleshooting of other problems within the FileNet environment.

### 2. Common Scenario

A user from the finance team attempts to add a new invoice document to IBM FileNet P8 but receives the **FNRAC1001W** warning message instead of completing the operation successfully. This might indicate that server-side logging has been activated, potentially due to an application or system misconfiguration.

### 3. Step-by-Step Troubleshooting

#### First Actionable Step:

**Verify User Permissions in ACCE**

1. **Log into IBM Connections Admin Console (ACCE).**
2. Navigate to the user permissions section.
3. Ensure that the finance team member has the necessary administrative rights to perform document operations.

*Why it's important:* This step is crucial because server trace logging can sometimes be enabled by an application with insufficient permissions, leading to this warning message being displayed.

#### Second Actionable Step:

**Check Server Logs**

1. Locate the **p8_server_error.log** file in the FileNet P8 installation directory.
2. Open the log file and search for entries related to trace logging.
3. Look for keywords such as "trace," "enabled," or "logging."

*Why it's important:* Checking server logs will provide insight into whether any application attempts have been made to enable tracing, which could explain why the warning is displayed.

#### Third Actionable Step:

**Review Class Definitions or Folder Security**

1. Access the **Object Store** and navigate through the class definitions.
2. Verify that folder security settings are correctly configured for users attempting document operations.
3. Check if any recent changes have been made to access permissions or ACLs (Access Control Lists).

*Why it's important:* Sometimes, enabling server trace logging might be associated with changing class definitions or security configurations that inadvertently affect user access rights.

### 4. Summary of Solutions

- **Revert Trace Logging:** If trace logging was enabled unintentionally, disable it to resolve the warning.
- **Adjust Permissions:** Ensure users have the correct permissions for document operations.
- **Review Security Configurations:** Verify and adjust any changes made to class definitions or folder security settings that might affect user access.

By following these steps, a junior administrator should be able to identify and resolve the **FNRAC1001W** error effectively.

-----------------------------------
CPU times: user 6.95 ms, sys: 4.75 ms, total: 11.7 ms
Wall time: 12.9 s


True

In [18]:
%%time
model_name="ibm/granite4:small-h"
load_model(model_name)

Sending a test message to the local model: 'ibm/granite4:small-h'...
Please wait, the first response can take a moment...

 Connection Successful! Model responded:
 "I am an AI language model dedicated to providing accurate and helpful information."
CPU times: user 3.58 ms, sys: 10.1 ms, total: 13.7 ms
Wall time: 5.66 s


In [19]:
%%time
generate_response(model_name, df, "FNRAC1001W")



 Found 'FNRAC1001W'. Generating guide with AI, please wait...
-----------------------------------


# Troubleshooting Guide: FNRAC1001W

## Understanding the Error

The error code **FNRAC1001W** with the message *"Server trace logging is enabled"* indicates that server trace logging has been activated. Trace logging is typically used for debugging purposes, allowing administrators to capture detailed information about system operations and potential issues.

For a junior administrator, this means:
- The system's performance might be impacted because capturing extensive logs can slow down processes.
- There could be disk space concerns as log files grow over time if not managed properly.
- It may lead to security or compliance issues since sensitive data could inadvertently be recorded in the logs.

## Common Scenario

Imagine a scenario where an administrator has been troubleshooting intermittent application crashes and, during this process, accidentally left server trace logging enabled. A few days later, they receive notifications that storage is running low on disk space due to the large volume of log files generated.

In another case, the finance team might be experiencing slow performance when adding new invoices into the system because the extensive logs are consuming resources needed for efficient operation.

## Step-by-Step Troubleshooting

### First Actionable Step: Verify Current Logging Status
1. **How-to**: Log in to the IBM FileNet Content Engine Administration Console (ACCE) and navigate to *Server > System Properties*.
2. **Why it's important**: Checking the current logging status helps you confirm whether server trace logs are indeed enabled, which directly addresses the error at hand.

### Second Actionable Step: Review Server Logs
1. **How-to**: Access the server log files typically located in `<install_directory>/logs/p8_server_error.log`.
2. **What to look for**: Search for keywords such as `"TRACE"`, `"LOGGING"`, or any entries that indicate verbose logging is active.
3. **Why it's important**: Reviewing these logs will provide insight into what processes were being logged and might reveal if there are other issues besides trace logging.

### Third Actionable Step: Adjust Logging Configuration
1. **How-to**:
   - In ACCE, go to *Server > System Properties*.
   - Locate the property related to logging (e.g., `P8_TRACE_LEVEL`).
   - Set this property to a lower level such as `ERROR` or `WARN`, depending on your needs for diagnostic information without excessive detail.
2. **Why it's important**: Adjusting the logging configuration helps reduce unnecessary data collection, freeing up system resources and addressing performance issues.

## Summary of Solutions

- **Revert Logging Level**: Change server trace logs back to a less verbose level like `ERROR` or `WARN`. This will help maintain necessary diagnostic capabilities without overloading system resources.
- **Monitor Disk Space Regularly**: Implement regular monitoring for disk space usage related to log files to prevent storage issues from impacting operations.
- **Review Log Policies**: Establish clear policies regarding when and how long server trace logging should be enabled. Ensure that it is disabled after troubleshooting unless explicitly needed.

By following these steps, you should be able to resolve the FNRAC1001W error effectively while maintaining optimal system performance and resource management.

-----------------------------------
CPU times: user 6.63 ms, sys: 19.7 ms, total: 26.4 ms
Wall time: 49.3 s


True

In [20]:
%%time
model_name="gpt-oss:20b"
load_model(model_name)


Sending a test message to the local model: 'gpt-oss:20b'...
Please wait, the first response can take a moment...

 Connection Successful! Model responded:
 "I'm ChatGPT, an AI language model here to help with questions, ideas, and tasks."
CPU times: user 3.31 ms, sys: 4.3 ms, total: 7.61 ms
Wall time: 13.2 s


In [21]:
%%time
generate_response(model_name, df, "FNRAC1001W")




 Found 'FNRAC1001W'. Generating guide with AI, please wait...
-----------------------------------


## Troubleshooting Guide: FNRAC1001W

### 1. Understanding the Error

**FNRAC1001W – WARN_TRACE_LOG_ENABLED**  
When you see this warning, the FileNet P8 repository is telling you that **trace logging is currently turned on**. Trace logging generates a large amount of diagnostic output (method entry/exit, SQL statements, internal state changes, etc.). While useful for debugging, it can:

- Consume disk space rapidly (logs grow to gigabytes in a short time).  
- Degrade system performance (IO overhead, CPU, memory).  
- Make it harder to spot real issues because the console is cluttered with routine messages.  

For a junior administrator, this message is a heads‑up that the repository may be running in a “debug” mode that you probably didn’t enable intentionally.

---

### 2. Common Scenario

> **Scenario:**  
> An analyst from the **Compliance** team logs into the FileNet **PowerCenter** console to run a routine report. Suddenly, the report fails and the server logs show:  
> ```
> FNRAC1001W: Server trace logging is enabled.
> ```  
> The analyst reports a slowdown and a “log file is filling up quickly” error.  
>  
> **Why this happens:**  
> In this environment, a recent **manual change** was made to enable `trace_logging` for troubleshooting an unrelated issue. The setting was left on, and now the entire repository is producing excessive trace data.

---

### 3. Step‑by‑Step Troubleshooting

> **Goal:** Identify where trace logging is enabled, turn it off (or reduce the level), and verify that the warning disappears.

---

#### 3.1 First Actionable Step – Verify Current Trace Configuration

| Sub‑Task | How to Do It | Why It Matters |
|----------|--------------|----------------|
| **Locate the Repository Configuration File** | 1. Connect to the FileNet **Engine** node via SSH/remote desktop.<br>2. Navigate to `<FileNetInstall>/Repository/<RepoName>/conf`.<br>3. Open `repository.properties` (or the relevant `engine.properties` file). | This file holds the `trace_logging` flag that controls whether trace logs are generated. |
| **Check the `trace_logging` Setting** | Search for `trace_logging=` in the file. Typical values: `true` or `false`. | If set to `true`, trace logging is on. |
| **Inspect `trace_level`** | Look for `trace_level=`. Common levels: `0` (none), `1` (basic), `2` (detailed). | Even if `trace_logging` is `false`, a high `trace_level` can still produce a lot of output. |

> **Command example (Linux):**  
> ```bash
> grep -i trace_logging <path_to_repo>/conf/repository.properties
> grep -i trace_level <path_to_repo>/conf/repository.properties
> ```

---

#### 3.2 Second Actionable Step – Disable or Reduce Trace Logging

| Sub‑Task | How to Do It | Why It Matters |
|----------|--------------|----------------|
| **Turn Off Trace Logging** | Edit the file and set `trace_logging=false`. | Stops all trace output immediately. |
| **Set a Lower Trace Level** | If you need some tracing, set `trace_level=0` (no trace) or `trace_level=1` (minimal). | Keeps logging at a manageable level. |
| **Save and Exit** | Ensure the file is saved with correct permissions (usually `root` or `filenet` user). | Prevents configuration from being overwritten. |
| **Restart the FileNet Engine** | ```bash
> cd <FileNetInstall>/Engine
> ./engine restart
> ``` | The Engine reads the new configuration only after restart. |  

> **Tip:** Use the **FileNet PowerCenter Admin console** to restart the repository instead of manual CLI if you’re more comfortable with a GUI.

---

#### 3.3 Third Actionable Step – Verify the Fix in Logs

| Sub‑Task | How to Do It | Why It Matters |
|----------|--------------|----------------|
| **Check the Server Error Log** | Open `<FileNetInstall>/logs/p8_server_error.log` (or `<RepoName>/logs/p8_server_error.log`). | This is where the warning originates. |
| **Search for the Warning** | ```bash
> grep -i FNRAC1001W p8_server_error.log
> ``` | The warning should no longer appear. |
| **Look for Other Trace Entries** | Search for keywords like `TRACE`, `DEBUG`, `INFO` to confirm that trace output has ceased. | Confirms that the change worked and performance should improve. |

> **Additional Step (Optional):**  
> If the repository is part of a **Cluster** or **Replication** environment, make sure the same changes are applied to **all nodes** to avoid inconsistent logging behavior.

---

#### 3.4 Optional Advanced Check – Verify PowerCenter Configuration

If the error keeps appearing after changing the engine config, the **PowerCenter** configuration might override it.

| Sub‑Task | How to Do It | Why It Matters |
|----------|--------------|----------------|
| **Open PowerCenter Admin Console** | Navigate to **FileNet PowerCenter Admin** (web UI). |
| **Check Repository Properties** | Under **Repositories → [Your Repo] → Properties**, look for **Trace Logging** toggle. |
| **Set Trace Logging Off** | Toggle to **Off** or set **Trace Level** to **0**. |
| **Restart PowerCenter Server** | Use the Admin console to restart, or run the script `p8_admin_restart.sh`. | Ensures that the change propagates to the repository process. |

---

### 4. Summary of Solutions

| Fix | When to Use | Effect |
|-----|-------------|--------|
| **Set `trace_logging=false` in `repository.properties`** | When trace logging is unnecessarily enabled. | Stops all trace logs, frees disk, improves performance. |
| **Reduce `trace_level` to 0 or 1** | When you still need some diagnostic output but want to limit volume. | Keeps logging minimal, retains essential traces. |
| **Restart FileNet Engine/PowerCenter** | After any configuration change. | Applies new settings; removes old warning from logs. |
| **Synchronize Cluster Nodes** | In a multi‑node environment. | Ensures consistent logging behavior across all nodes. |
| **Check PowerCenter Admin Console** | When the warning persists after engine change. | Confirms PowerCenter isn’t overriding the setting. |

> **Key Takeaway:**  
> FNRAC1001W is a warning that trace logging is active. Turning it off (or down to an acceptable level) and restarting the repository will eliminate the warning and restore normal operation. Always monitor disk usage after changes to confirm that logs no longer grow uncontrollably.

-----------------------------------
CPU times: user 5.54 ms, sys: 6.24 ms, total: 11.8 ms
Wall time: 46.3 s


True